## Data Pre-Processing

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("data-4.xlsx")  

In [3]:
df.head()

,User_id,Name,Quiz_id,Attempts,Score,Response
0,656,Avani Sharma,4,1,9.0,"{""1"":{""quiz_question_id"":31,""selected_option"":..."
1,6890,geetha,4,1,9.0,"{""1"":{""quiz_question_id"":31,""selected_option"":..."
2,6891,aashish,4,1,9.0,"{""1"":{""quiz_question_id"":31,""selected_option"":..."
3,367,Mridhul Khajuria,4,1,5.0,"{""1"":{""quiz_question_id"":31,""selected_option"":..."
4,367,Mridhul Khajuria,4,2,9.0,"{""1"":{""quiz_question_id"":31,""selected_option"":..."


In [4]:
df.size

3386532

In [5]:
df.shape

(564422, 6)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564422 entries, 0 to 564421
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User_id   564422 non-null  int64  
 1   Name      564422 non-null  object 
 2   Quiz_id   564422 non-null  int64  
 3   Attempts  564422 non-null  int64  
 4   Score     563763 non-null  float64
 5   Response  564422 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 25.8+ MB


## Parsing Json to making columns

In [7]:
import json

df['Response'] = df['Response'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

In [8]:
processed_rows = []

In [9]:
for idx, row in df.iterrows():
    base_data = {
        'User_id': row['User_id'],
        'Name': row['Name'],
        'Quiz_id': row['Quiz_id'],
        'Attempts': row['Attempts'],
        'Total_Score': row['Score']  
    }
    
    response_dict = row['Response']
    for question_key, answer in response_dict.items():
        answer['Question_Key'] = question_key
        
        combined_row = {**base_data, **answer}
        processed_rows.append(combined_row)

In [10]:
df_processed = pd.DataFrame(processed_rows)

In [11]:
print(df_processed.head())

   User_id          Name  Quiz_id  Attempts  Total_Score  quiz_question_id  \
0      656  Avani Sharma        4         1          9.0                31   
1      656  Avani Sharma        4         1          9.0                32   
2      656  Avani Sharma        4         1          9.0                33   
3      656  Avani Sharma        4         1          9.0                34   
4      656  Avani Sharma        4         1          9.0                35   

                                     selected_option  attempts  \
0                Yes, it was fun and we learnt a lot         1   
1                                               Both         1   
2                                 A-3\nB-2\nC-4\nD-1         1   
3                                            Process         1   
4  Some problems may have both types and some may...         1   

                                            question  \
0  Were you able to finish the Workbook for Modul...   
1  What kind of soluti

## Other Basic Pre-Processing

In [12]:
df_processed.columns

Index(['User_id', 'Name', 'Quiz_id', 'Attempts', 'Total_Score',
       'quiz_question_id', 'selected_option', 'attempts', 'question',
       'correct_answer', 'level', 'question_no', 'is_correct', 'score',
       'Question_Key'],
      dtype='object')

In [13]:
df_processed.shape

(5639003, 15)

In [14]:
df_processed.isnull().sum()

User_id                 0
Name                    0
Quiz_id                 0
Attempts                0
Total_Score          6110
quiz_question_id        0
selected_option         0
attempts                0
question                0
correct_answer          0
level                   0
question_no             0
is_correct              0
score               61168
Question_Key            0
dtype: int64

#### Dropped na values for higher accuracy

In [15]:
df_cleaned = df_processed.dropna()

In [16]:
df_processed.shape

(5639003, 15)

#### We have two similar columns Attempts and attempts

In [17]:
df_cleaned = df_cleaned.drop(columns=['attempts'])

#### Striping Whitespace 

In [18]:
str_cols = ['Name', 'selected_option', 'question', 'correct_answer', 'level']
df_cleaned[str_cols] = df_cleaned[str_cols].apply(lambda x: x.str.strip())

#### droping duplicate values

In [19]:
df_cleaned = df_cleaned.drop_duplicates()

In [20]:
df_cleaned.shape

(5577829, 14)

## Exporting to CSV

In [21]:
df_cleaned.to_csv('df_cleaned_4.csv', index=False)

In [22]:
# Load the full cleaned data (don't print the whole thing!)
df_final = pd.read_csv('df_cleaned.csv')

print("✅ Data loaded successfully!")
print("🔢 Shape:", df_final.shape)
print("👀 Preview:")
print(df_final.head())  # Only shows first 5 rows


✅ Data loaded successfully!
🔢 Shape: (5577829, 14)
👀 Preview:
   User_id          Name  Quiz_id  Attempts  Total_Score  quiz_question_id  \
0      656  Avani Sharma        4         1          9.0                31   
1      656  Avani Sharma        4         1          9.0                32   
2      656  Avani Sharma        4         1          9.0                33   
3      656  Avani Sharma        4         1          9.0                34   
4      656  Avani Sharma        4         1          9.0                35   

                                     selected_option  \
0                Yes, it was fun and we learnt a lot   
1                                               Both   
2                                 A-3\nB-2\nC-4\nD-1   
3                                            Process   
4  Some problems may have both types and some may...   

                                            question  \
0  Were you able to finish the Workbook for Modul...   
1  What kind of solu